In [16]:
!pip install lightgbm
!pip install catboost
!pip install scikit-learn
!pip install pandas
!pip install transformers
!pip install torch
!pip install nltk
!pip install numpy
!pip install pymystem3

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.2 MB/s eta 0:00:0000:010:01
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 14.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 28.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 28.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 30.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 30.8 MB/s eta 

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem

# Укажите путь к вашему файлу CSV
file_path = 'train_data.csv'

# Чтение файла CSV с помощью Pandas и вывод первых 10 строк
df = pd.read_csv(file_path)
print(df.head(10))

# Загрузка русских стоп-слов
nltk.download('stopwords')
nltk.download('punkt')

/home/datadisk/jupyter-admin/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


             timestamp                            question_1  \
0  2024-04-01 08:30:00               Основы программирования   
1  2024-04-01 10:00:00               Основы программирования   
2  2024-04-01 15:00:00               Основы программирования   
3  2024-04-01 17:00:00               Основы программирования   
4  2024-04-02 09:00:00               Основы программирования   
5  2024-04-02 10:00:00               Основы программирования   
6  2024-04-02 11:00:00               Основы программирования   
7  2024-04-02 18:45:00  Продвинутые техники программирования   
8  2024-04-03 12:00:00  Продвинутые техники программирования   
9  2024-04-03 15:00:00  Продвинутые техники программирования   

                                          question_2  \
0  Интерактивный подход к изучению основ C# был о...   
1  Подробное объяснение логических операторов в J...   
2  Введение в программирование на Python было про...   
3  Понятное изложение основных понятий, хорошая с...   
4  Кларность и 

[nltk_data] Downloading package stopwords to /home/datadisk/jupyter-
[nltk_data]     admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/datadisk/jupyter-
[nltk_data]     admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Загрузка стоп-слов и инициализация Mystem для лемматизации
stop_words = set(stopwords.words('russian'))
mystem = Mystem()

# Инициализация токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [4]:
import numpy as np

# Функция для предобработки текста на русском языке
def preprocess_text_russian(text):
    # Токенизация текста
    tokens = word_tokenize(text)
    # Удаление русских стоп-слов
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Лемматизация токенов
    lemmatized_tokens = [mystem.lemmatize(token)[0] for token in filtered_tokens]
    # Объединение лемматизированных токенов обратно в текст
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

# Функция для преобразования текста с помощью BERT
def bert_embeddings(text):
    # Токенизация текста
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # Получение эмбеддингов текста от модели BERT
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Используем средний пулинг для получения одного эмбеддинга для всего текста
    return embeddings.numpy()

In [5]:
# Преобразование каждого отзыва в столбцах 'question_2', 'question_3', 'question_4', 'question_5' с помощью BERT
for column in ['question_2', 'question_3', 'question_4', 'question_5']:
    df[column + '_preprocessed'] = df[column].apply(preprocess_text_russian)
    df[column + '_bert_embeddings'] = df[column + '_preprocessed'].apply(bert_embeddings)

/home/datadisk/jupyter-admin/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
print(df.head(10))

             timestamp                            question_1  \
0  2024-04-01 08:30:00               Основы программирования   
1  2024-04-01 10:00:00               Основы программирования   
2  2024-04-01 15:00:00               Основы программирования   
3  2024-04-01 17:00:00               Основы программирования   
4  2024-04-02 09:00:00               Основы программирования   
5  2024-04-02 10:00:00               Основы программирования   
6  2024-04-02 11:00:00               Основы программирования   
7  2024-04-02 18:45:00  Продвинутые техники программирования   
8  2024-04-03 12:00:00  Продвинутые техники программирования   
9  2024-04-03 15:00:00  Продвинутые техники программирования   

                                          question_2  \
0  Интерактивный подход к изучению основ C# был о...   
1  Подробное объяснение логических операторов в J...   
2  Введение в программирование на Python было про...   
3  Понятное изложение основных понятий, хорошая с...   
4  Кларность и 

In [7]:
# Длина значения в первой строке столбца 'question_2_bert_embeddings'
length_value = len(df.loc[0, 'question_2_bert_embeddings'])
print("Длина значения в первой строке столбца 'question_2_bert_embeddings':", length_value)

# Тип данных значения в первой строке столбца 'question_2_bert_embeddings'
data_type = type(df.loc[0, 'question_2_bert_embeddings'])
print("Тип данных значения в первой строке столбца 'question_2_bert_embeddings':", data_type)

# Если вложенный массив, вывести количество массивов внутри и количество элементов в каждом массиве
if isinstance(df.loc[0, 'question_2_bert_embeddings'], np.ndarray):
    nested_array = df.loc[0, 'question_2_bert_embeddings']
    num_arrays = len(nested_array)
    num_elements = [len(arr) for arr in nested_array]
    print("Количество массивов внутри:", num_arrays)
    print("Количество элементов в каждом массиве:", num_elements)

Длина значения в первой строке столбца 'question_2_bert_embeddings': 1
Тип данных значения в первой строке столбца 'question_2_bert_embeddings': <class 'numpy.ndarray'>
Количество массивов внутри: 1
Количество элементов в каждом массиве: [768]


In [8]:
# Список столбцов, для которых нужно выполнить преобразование
columns_to_flatten = ['question_2_bert_embeddings', 'question_3_bert_embeddings', 'question_4_bert_embeddings', 'question_5_bert_embeddings']

# Проход по каждому столбцу
for column in columns_to_flatten:
    # Проход по каждой строке в столбце
    for index, value in df[column].items():
        # Проверка, является ли значение вложенным массивом numpy.ndarray
        if isinstance(value, np.ndarray):
            # Преобразование вложенного массива в одномерный массив
            flat_array = value.flatten()
            # Присвоение нового значения ячейке в DataFrame
            df.at[index, column] = flat_array

# Вывод DataFrame для проверки изменений
print(df)

               timestamp               question_1  \
0    2024-04-01 08:30:00  Основы программирования   
1    2024-04-01 10:00:00  Основы программирования   
2    2024-04-01 15:00:00  Основы программирования   
3    2024-04-01 17:00:00  Основы программирования   
4    2024-04-02 09:00:00  Основы программирования   
..                   ...                      ...   
98   2024-04-24 10:22:00  Основы программирования   
99   2024-04-24 18:00:00  Новейшие тенденции в IT   
100  2024-04-25 20:00:00  Новейшие тенденции в IT   
101     27.04.2024:19:00       Java api браузеров   
102     27.04.2024:19:00       Java api браузеров   

                                            question_2  \
0    Интерактивный подход к изучению основ C# был о...   
1    Подробное объяснение логических операторов в J...   
2    Введение в программирование на Python было про...   
3    Понятное изложение основных понятий, хорошая с...   
4    Кларность и структура материала по основам Jav...   
..             

In [9]:
# Создание нового столбца для объединенных значений
df['combined_embeddings'] = df.apply(lambda row: np.concatenate([row['question_2_bert_embeddings'],
                                                                 row['question_3_bert_embeddings'],
                                                                 row['question_4_bert_embeddings'],
                                                                 row['question_5_bert_embeddings']]), axis=1)

# Вывод DataFrame для проверки изменений
print(df)

               timestamp               question_1  \
0    2024-04-01 08:30:00  Основы программирования   
1    2024-04-01 10:00:00  Основы программирования   
2    2024-04-01 15:00:00  Основы программирования   
3    2024-04-01 17:00:00  Основы программирования   
4    2024-04-02 09:00:00  Основы программирования   
..                   ...                      ...   
98   2024-04-24 10:22:00  Основы программирования   
99   2024-04-24 18:00:00  Новейшие тенденции в IT   
100  2024-04-25 20:00:00  Новейшие тенденции в IT   
101     27.04.2024:19:00       Java api браузеров   
102     27.04.2024:19:00       Java api браузеров   

                                            question_2  \
0    Интерактивный подход к изучению основ C# был о...   
1    Подробное объяснение логических операторов в J...   
2    Введение в программирование на Python было про...   
3    Понятное изложение основных понятий, хорошая с...   
4    Кларность и структура материала по основам Jav...   
..             

In [10]:
# Получение значения из ячейки столбца 'combined_embeddings' в первой строке
value = df.at[0, 'combined_embeddings']

# Подсчет количества элементов в массиве в этой ячейке
num_elements = len(value)

# Вывод количества элементов
print("Количество элементов в ячейке 'combined_embeddings':", num_elements)
X = df['combined_embeddings']
X

Количество элементов в ячейке 'combined_embeddings': 3072


0      [0.5780365, -0.7246146, 0.48258838, 0.24729064...
1      [0.3808728, -0.6132502, 0.3899155, 0.49545494,...
2      [0.35725695, -0.81302744, 0.24997342, 0.441849...
3      [0.105378784, -0.28539807, 0.8758947, -0.10220...
4      [0.61808157, -0.4399408, 0.85798264, -0.121556...
                             ...                        
98     [-0.10421173, -0.30873016, 0.6809484, 0.347523...
99     [0.14024127, -0.58888584, 0.30238628, -0.23862...
100    [0.17956083, -0.20090167, 0.45330378, 0.079756...
101    [-0.10814951, -0.19484048, 0.6745561, 0.143788...
102    [0.14148219, -0.5173893, 0.55626166, 0.2244017...
Name: combined_embeddings, Length: 103, dtype: object

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

# Преобразование столбца 'combined_embeddings' в список массивов
embeddings_list = df['combined_embeddings'].tolist()

# Объединение массивов в один двумерный массив
X = np.vstack(embeddings_list)

# Выбор целевых переменных
y = df[['is_relevant', 'object', 'is_positive']]

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Создание моделей для каждой целевой переменной
model_is_relevant = LGBMClassifier(random_state=42)
model_object = LGBMClassifier(random_state=42)
model_is_positive = LGBMClassifier(random_state=42)

# Обучение моделей
model_is_relevant.fit(X_train, y_train['is_relevant'])
model_object.fit(X_train, y_train['object'])
model_is_positive.fit(X_train, y_train['is_positive'])

# Прогнозы на тестовом наборе данных
y_pred_is_relevant = model_is_relevant.predict(X_test)
y_pred_object = model_object.predict(X_test)
y_pred_is_positive = model_is_positive.predict(X_test)

# Оценка производительности моделей
f1_is_relevant = f1_score(y_test['is_relevant'], y_pred_is_relevant, average='binary')
f1_object = f1_score(y_test['object'], y_pred_object, average='weighted')
f1_is_positive = f1_score(y_test['is_positive'], y_pred_is_positive, average='binary')

# Вывод метрик F1 для каждой целевой переменной
print(f"F1 Score for is_relevant: {f1_is_relevant}")
print(f"F1 Score for object: {f1_object}")
print(f"F1 Score for is_positive: {f1_is_positive}")

[LightGBM] [Info] Number of positive: 81, number of negative: 11
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 93251
[LightGBM] [Info] Number of data points in the train set: 92, number of used features: 3072
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.880435 -> initscore=1.996554
[LightGBM] [Info] Start training from score 1.996554
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [24]:
from joblib import dump

# Сохранение моделей в файлы
dump(model_is_relevant, 'model_is_relevant.joblib')
dump(model_object, 'model_object.joblib')
dump(model_is_positive, 'model_is_positive.joblib')

['model_is_positive.joblib']